In [1]:
# OI Analysis & Max Pain

This notebook:
- Fetches option chain data from ICICI Breeze API
- Calculates PCR (OI & Volume) and Max Pain
- Logs results to Excel
- Visualizes OI distribution and total loss curves

Note: Requires valid Breeze API credentials in `scripts/breeze_client.py` (local only).


SyntaxError: invalid character '⚠' (U+26A0) (710382391.py, line 9)

In [2]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output
from datetime import datetime
import sys, os

# Add project root so we can import scripts/
sys.path.append(os.path.abspath(".."))

# Import reusable functions from scripts/oi_analysis.py
from scripts.oi_analysis import (
    fetch_option_chain_data,
    calculate_pcr_max_pain,
    append_to_excel,
    plot_open_interest,
    plot_total_loss,
    fetch_underlying_ltp
)


In [3]:
def interactive_option_chain():
    stock_input = widgets.Text(
        description="Stock Code:",
        placeholder="Enter stock name (e.g., NIFTY, BANKNIFTY)",
        style={"description_width": "initial"}
    )
    expiry_input = widgets.DatePicker(
        description="Select Expiry Date:",
        style={"description_width": "initial"}
    )
    fetch_button = widgets.Button(
        description="Fetch & Visualize",
        button_style="success"
    )
    output_box = widgets.Output()

    def on_fetch_button_clicked(_):
        with output_box:
            clear_output()
            stock_code = stock_input.value.strip().upper()
            expiry_date = expiry_input.value
            if not stock_code or not expiry_date:
                print("⚠️ Please provide valid inputs.")
                return

            expiry_date_iso = expiry_date.isoformat() + "T06:00:00.000Z"
            data = fetch_option_chain_data(stock_code, expiry_date_iso)

            if not data.empty:
                results = calculate_pcr_max_pain(data)
                plot_total_loss(data, stock_code)
                now = datetime.now()
                ltp = fetch_underlying_ltp(stock_code)

                output = pd.DataFrame({
                    "Date": [now.date()],
                    "Time": [now.strftime("%H:%M:%S")],
                    "PCR OI": [results["PCR OI"]],
                    "PCR Volume": [results["PCR Volume"]],
                    "Max Pain": [results["Max Pain"]],
                    "LTP": [ltp]
                })
                display(output)

                append_to_excel(output, "data/Indicators_Log.xlsx")
                plot_open_interest(data, stock_code)
            else:
                print("❌ No data retrieved for the given inputs.")

    fetch_button.on_click(on_fetch_button_clicked)
    display(widgets.VBox([stock_input, expiry_input, fetch_button, output_box]))


In [4]:
interactive_option_chain()
